# ddim官方源码分析
[ddim](https://github.com/ermongroup/ddim)

In [6]:
import numpy as np
import torch

device="cuda"

$\beta_t \in(0,1)$ 的计算方式

取值范围在(0,1),随着time_step逐步递增

In [14]:
# $\beta_t \in(0,1)$ 的计算方式
def get_beta_schedule(beta_schedule, *, beta_start, beta_end, num_diffusion_timesteps):
    print("Use:",beta_schedule)
    def sigmoid(x):
        return 1 / (np.exp(-x) + 1)

    if beta_schedule == "quad":
        betas = (
            np.linspace(
                beta_start ** 0.5,
                beta_end ** 0.5,
                num_diffusion_timesteps,
                dtype=np.float64,
            )
            ** 2
        )
    elif beta_schedule == "linear":
        betas = np.linspace(
            beta_start, beta_end, num_diffusion_timesteps, dtype=np.float64
        )
    elif beta_schedule == "const":
        betas = beta_end * np.ones(num_diffusion_timesteps, dtype=np.float64)
    elif beta_schedule == "jsd":  # 1/T, 1/(T-1), 1/(T-2), ..., 1
        betas = 1.0 / np.linspace(
            num_diffusion_timesteps, 1, num_diffusion_timesteps, dtype=np.float64
        )
    elif beta_schedule == "sigmoid":
        betas = np.linspace(-6, 6, num_diffusion_timesteps)
        betas = sigmoid(betas) * (beta_end - beta_start) + beta_start
    else:
        raise NotImplementedError(beta_schedule)
    assert betas.shape == (num_diffusion_timesteps,)
    return betas


beta_start = 0.0001
beta_end = 0.02
num_diffusion_timesteps = 100 #论文是1000，为了方便测试观察取100

beta_schedule = "linear"
betas = get_beta_schedule(
            beta_schedule=beta_schedule,
            beta_start=beta_start,
            beta_end=beta_end,
            num_diffusion_timesteps=num_diffusion_timesteps,
        )
print(betas)


beta_schedule = "const"
betas = get_beta_schedule(
            beta_schedule=beta_schedule,
            beta_start=beta_start,
            beta_end=beta_end,
            num_diffusion_timesteps=num_diffusion_timesteps,
        )
print(betas)


beta_schedule = "jsd"
betas = get_beta_schedule(
            beta_schedule=beta_schedule,
            beta_start=beta_start,
            beta_end=beta_end,
            num_diffusion_timesteps=num_diffusion_timesteps,
        )
print(betas)


beta_schedule = "sigmoid"
betas = get_beta_schedule(
            beta_schedule=beta_schedule,
            beta_start=beta_start,
            beta_end=beta_end,
            num_diffusion_timesteps=num_diffusion_timesteps,
        )
print(betas)

Use: linear
[0.0001     0.00030101 0.00050202 0.00070303 0.00090404 0.00110505
 0.00130606 0.00150707 0.00170808 0.00190909 0.0021101  0.00231111
 0.00251212 0.00271313 0.00291414 0.00311515 0.00331616 0.00351717
 0.00371818 0.00391919 0.0041202  0.00432121 0.00452222 0.00472323
 0.00492424 0.00512525 0.00532626 0.00552727 0.00572828 0.00592929
 0.0061303  0.00633131 0.00653232 0.00673333 0.00693434 0.00713535
 0.00733636 0.00753737 0.00773838 0.00793939 0.0081404  0.00834141
 0.00854242 0.00874343 0.00894444 0.00914545 0.00934646 0.00954747
 0.00974848 0.00994949 0.01015051 0.01035152 0.01055253 0.01075354
 0.01095455 0.01115556 0.01135657 0.01155758 0.01175859 0.0119596
 0.01216061 0.01236162 0.01256263 0.01276364 0.01296465 0.01316566
 0.01336667 0.01356768 0.01376869 0.0139697  0.01417071 0.01437172
 0.01457273 0.01477374 0.01497475 0.01517576 0.01537677 0.01557778
 0.01577879 0.0159798  0.01618081 0.01638182 0.01658283 0.01678384
 0.01698485 0.01718586 0.01738687 0.01758788 0.0177

In [15]:
betas  = torch.from_numpy(betas).float().to(device)
num_timesteps = betas.shape[0]
alphas = 1.0 - betas
alphas_cumprod = alphas.cumprod(dim=0)
alphas_cumprod_prev = torch.cat(
    [torch.ones(1).to(device), alphas_cumprod[:-1]], dim=0
) #这个地方直接在index=0的位置 放了1，做补位。

print("alphas:",alphas)
print("alphas_cumprod:",alphas_cumprod)
print("alphas_cumprod_prev:",alphas_cumprod_prev)

alphas: tensor([0.9999, 0.9998, 0.9998, 0.9998, 0.9998, 0.9998, 0.9998, 0.9998, 0.9998,
        0.9998, 0.9997, 0.9997, 0.9997, 0.9997, 0.9996, 0.9996, 0.9996, 0.9995,
        0.9995, 0.9994, 0.9994, 0.9993, 0.9992, 0.9991, 0.9990, 0.9989, 0.9988,
        0.9987, 0.9985, 0.9984, 0.9982, 0.9980, 0.9978, 0.9975, 0.9973, 0.9970,
        0.9967, 0.9963, 0.9959, 0.9955, 0.9951, 0.9947, 0.9942, 0.9937, 0.9931,
        0.9926, 0.9920, 0.9914, 0.9909, 0.9903, 0.9896, 0.9890, 0.9885, 0.9879,
        0.9873, 0.9868, 0.9862, 0.9857, 0.9852, 0.9848, 0.9844, 0.9840, 0.9836,
        0.9832, 0.9829, 0.9826, 0.9824, 0.9821, 0.9819, 0.9817, 0.9815, 0.9814,
        0.9812, 0.9811, 0.9810, 0.9809, 0.9808, 0.9807, 0.9806, 0.9805, 0.9805,
        0.9804, 0.9804, 0.9803, 0.9803, 0.9803, 0.9802, 0.9802, 0.9802, 0.9802,
        0.9801, 0.9801, 0.9801, 0.9801, 0.9801, 0.9801, 0.9801, 0.9801, 0.9801,
        0.9800], device='cuda:0')
alphas_cumprod: tensor([0.9999, 0.9997, 0.9995, 0.9994, 0.9992, 0.9990, 0.9988

In [ ]:
1.训练和DDPM保持不变

2.采用的公式变了，原来需要1000 steps，现在50个step就可以了，也就是中间可以跳过去。
